## Clustering

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import nyu_dataset
import pix2pixHD_model
from torch.utils.data import DataLoader 
from sklearn.cluster import KMeans
from collections import defaultdict

In [ ]:
_cuda = 2

In [ ]:
encoder = pix2pixHD_model.FeatureEncoder(3, 3, instance_norm=True).cuda(_cuda)
encoder.load_state_dict(torch.load('./pix2pixHD_newsegm/models/encoder_170_0'))

In [ ]:
segmentation_dataset_train = nyu_dataset.SegmentationDataset(
        transforms=nyu_dataset.SegmentationTransform(resize=False), 
        use_instance_segmentation=True)
data_loader_train = DataLoader(segmentation_dataset_train, batch_size=1, shuffle=True, num_workers=1)

In [ ]:
dictionary = {}


for image_batch, segmentation_batch, inst_batch, border_batch in data_loader_train:
    image = Variable(image_batch.cuda(_cuda), requires_grad=False)
    segm = Variable(segmentation_batch.cuda(_cuda), requires_grad=False)
    inst = inst_batch.cuda(_cuda)
  
    encoded = encoder.forward(image, inst)

    array_segm = segm.data.cpu().numpy()
    array_encoded = encoded.data.cpu().numpy()
    classes = range(array_segm.shape[1])

    for k in classes:   
        idx = np.argwhere(array_segm[0,k,...]==1)
        for x, y in idx:
            feature_array = tuple(array_encoded[0, :, x, y].tolist())
            dictionary.setdefault(k,set()).add(feature_array)

In [ ]:
cluster_centers = defaultdict(list)
for k, features in dictionary.items():
    kmeans = KMeans(n_clusters = 10).fit([list(feat) for feat in features])
    k_centers = kmeans.cluster_centers_
    cluster_centers[k].append(k_centers)

## Number of instance and number of classes

In [ ]:
i =0 
for image, segm, inst, bord in data_loader_val:
    image = Variable(image.cuda(_cuda), requires_grad=False, volatile=True)
    segm = Variable(segm.cuda(_cuda), requires_grad=False, volatile=True)
    inst = inst.cuda(_cuda)
    i += 1
    if i == 25:
        break

In [ ]:
classes_map = np.argmax(segm[0].data.cpu().numpy(), axis=0)

instance = inst.cpu().numpy()

print(len(np.unique(classes_map)))
print(len(np.unique(instance)))